## Word2Vec using NLTK/Glove/GoogleNews

#### Import Libraries

In [76]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from IPython.display import Image # To view image from location/url
import keras
import nltk
import logging
import multiprocessing
import re
import os
import gensim
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import gensim.models.keyedvectors as word2vec
from nltk.corpus import brown, movie_reviews, treebank
#nltk.download('brown')
#nltk.download('movie_reviews')
#nltk.download('treebank')
#nltk.download('punkt')

# Input & Output files path
inputDir = "./data/"
outputDir = "./data/outDir/"
inputDir_BigFiles = "E:/BigFiles/"
outputDir_Bigfiles = "E:/BigFiles/outDir/"

#### Load NLTK model

In [54]:
brown_model = Word2Vec(brown.sents())
movie_reviews_model = Word2Vec(movie_reviews.sents())
treebank_model = Word2Vec(treebank.sents())
brown_model.save(outputDir+"brown_model")
size = str(round((os.path.getsize(outputDir+'brown_model')/1000),1))
print('Brown corpus model saved at '+outputDir+ '\nSize of file :'+size+ ' KB' )

Brown corpus model saved at ./data/outDir/
Size of file :19197.8 KB


#### Test NLTK model

In [55]:
# Brown
print(brown_model.most_similar('money', topn=5))
#find the odd one out
print (brown_model.doesnt_match("breakfast cereal dinner lunch".split()))
#vector representation of word human
#print (brown_model["human"])

[('care', 0.9140259027481079), ('chance', 0.9021503925323486), ('job', 0.8922946453094482), ('trouble', 0.8632405996322632), ('easy', 0.860753059387207)]
cereal


In [56]:
# Movie Reviews
print(movie_reviews_model.most_similar('money', topn=5))
#find the odd one out
print (movie_reviews_model.doesnt_match("girl boy her movie".split()))
#vector representation of word human
#print (movie_reviews_model["human"])

[('him', 0.7941465377807617), ('someone', 0.7558401823043823), ('sleep', 0.7459496259689331), ('attention', 0.7414337396621704), ('brain', 0.737573504447937)]
movie


In [57]:
# Treebank
print(treebank_model.most_similar('money', topn=5))
#find the odd one out
print (treebank_model.doesnt_match("financial money earning fat".split()))
#vector representation of word human
#print (treebank_model["human"])

[('all', 0.9999011158943176), ('only', 0.9998963475227356), ('new', 0.9998949766159058), ('some', 0.999890923500061), ("'", 0.9998877048492432)]
money


#### Load Glove model

In [59]:
# Convert glove_input_file in GloVe format into word2vec_output_file in word2vec format.
gensim.scripts.glove2word2vec.glove2word2vec(glove_input_file = inputDir_BigFiles+"glove.6B/glove.6B.50d.txt", 
                                             word2vec_output_file = outputDir_Bigfiles+"word2vec_glove_file")
size = str(round((os.path.getsize(outputDir_Bigfiles+"word2vec_glove_file")/1000000),1))
print('Glove model size :'+size+ ' MB')
# Load GloVe: Global Vectors for Word Representation Word2Vec model.
glove_model = word2vec.KeyedVectors.load_word2vec_format(outputDir_Bigfiles+"word2vec_glove_file")

Glove model size :171.4 MB


#### Test Glove model

In [67]:
print(glove_model.most_similar('messi', topn=5))
#find the odd one out
print (glove_model.doesnt_match("cricket football sports lunch".split()))

[('ronaldinho', 0.9371346235275269), ('rivaldo', 0.9020113348960876), ('ronaldo', 0.8994787931442261), ('figo', 0.8895716667175293), ("eto'o", 0.8803616166114807)]
lunch


#### Load Google Word2Vec model

In [61]:
# Load Google's pre-trained Word2Vec model.
google_model = word2vec.KeyedVectors.load_word2vec_format \
(inputDir_BigFiles+'GoogleNews_w2v/GoogleNews-vectors-negative300.bin', binary=True) 
size = str(round((os.path.getsize(inputDir_BigFiles+'GoogleNews_w2v/GoogleNews-vectors-negative300.bin')/1000000),1))
print('Google model size :'+size+ ' MB')

Google model size :3644.3 MB


#### Test Google Word2Vec model

In [69]:
print(google_model.most_similar('cricket', topn=5))
#find the odd one out
print (google_model.doesnt_match("cricket football sports lunch kohli".split()))

[('artificial', 0.6219794154167175), ('artifical', 0.5527142882347107), ('Artifical', 0.5464799404144287), ('Plastic', 0.5309567451477051), ('Synthetic', 0.47686296701431274)]
lunch


#### Visualizing Word2Vec

TSNE is pretty useful when it comes to visualizing similarity between objects. It works by taking a group of high-dimensional  vocabulary word feature vectors, then compresses them down to 2-dimensional x,y coordinate pairs. The idea is to keep similar words close together on the plane, while maximizing the distance between dissimilar words.

In [ ]:
### Define function to build the model

In [81]:
def tsne_plot(sentences=None, size=100, window=5, min_count=5):
#'''Creates and TSNE model and plots it
#size: (default 100) The number of dimensions of the embedding, 
#e.g. the length of the dense vector to represent each token (word).
#window: (default 5) The maximum distance between a target word and words around the target word.
#min_count: (default 5) The minimum count of words to consider when training the model; words with an occurrence less than this count will be ignored.
#workers: (default 3) The number of threads to use while training.
#sg: (default 0 or CBOW) The training algorithm, either CBOW (0) or skip gram (1).
#'''
    model = Word2Vec(sentences,size = size,window=window, min_count=min_count)
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
# A more selective model
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=500, workers=4)
#normal
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=200, workers=4) 
# A less selective model
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=100, workers=4)

In [ ]:
tsne_plot(sentences=brown.sents(), size=10, window=2, min_count=5)